In [ ]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date
import pandas as pd
import calendar
import numpy.ma as ma

import createFolder as cF
import plot_vertical as pvert
import plot_sfc_spaghetti as spagh
import fill_values as fv
import save_fig as sF
import calc_date as cd



import os




In [ ]:
# 16th doesnt exist, 17-19 have different shapes
t = [
#    1, 2, 3, 4, 5, 6 , 7, 8, 9,10, \
 #   11,12,13,14,15,    
  17,18,19,20, 21,22,23, 
    24,25,26,27,28,29,30, 31
    ]
year = 2016
month = 12
tid = '00'
savefig = 0  # 1 = yes, 0 = no

station = 'Haukeliseter'

In [ ]:
var_name = 'snowfall_amount'
unit = '[g/m$^2$]'
#var_name = 'rainfall_amount'
#var_name = 'graupelfall_amount'
#var_name = 'atmosphere_cloud_condensed_water_content'
#var_name = 'atmosphere_cloud_ice_content'
#var_name = 'air_temperature'
var_name_ml = '%s_ml' %(var_name)


ncdir_ml = '../../MEPS_data/%s/ml/%s' %(station,var_name_ml)
figdir = '../../MEPS_fig/%s' %(var_name_ml)
form = 'png'
cF.createFolder('%s' %(figdir))

In [ ]:
for day in t:
    if day < 10:
        day = '0%s' %(str(day))
    
    var_name_ml = '%s_ml' %(var_name)

## open netCDF 
    fn_ml = dict()
    time_ml = dict()    
    for ens_memb  in range(0,10):
        fn_ml[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_ml,year,month,day,tid,ens_memb))
# Read in time to know initialisation time (plotting)    
        time_ml[ens_memb] = fn_ml[ens_memb].variables['time']
        time_ml[ens_memb] = fv.fill_nan(time_ml[ens_memb][:])
        

    ini_day = datetime.datetime.utcfromtimestamp(time_ml[0][0]).day   # day of initialisation
    hh = datetime.datetime.utcfromtimestamp(time_ml[0][0]).hour       # first hour of initialisation?
    calday, calmon = cd.get_dayname(year, month, ini_day)

    
    
    
    lead_time = dict()
    snow_amount_ml = dict()
    thickness_ml = dict()
    h = dict()
    td = dict()
    height = dict()
    time = dict()
    maxim = []

    for ens_memb in range(0,10):
# Read in the variable name which should be plotted  
        snow_amount_ml[ens_memb] = fn_ml[ens_memb].variables[var_name_ml]

# Read in the thickness of a layer
        thickness_ml[ens_memb] = fn_ml[ens_memb].variables['layer_thickness']
# create an array with time of the shape of the variable
#        lead_time = np.arange(0,time_ml[0].size)
        lead_time = np.arange(0,snow_amount_ml[ens_memb].shape[0])
        model_level = np.arange(0,snow_amount_ml[ens_memb].shape[1])
        td,Y = np.meshgrid(lead_time,model_level)

# substitute missing values with nan       
        snow_amount_ml[ens_memb] = fv.fill_nan(snow_amount_ml[ens_memb][:])
        thickness_ml[ens_memb] = fv.fill_nan(thickness_ml[ens_memb][:])
        

# calculate the height from the thickness
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        
        th_arr = np.nansum([ thickness[:,0], thickness[:,(0+1)]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i]], axis = 0)
            h_above.append(th_arr)
        h_above = np.transpose(h_above)

        # get only valuable values not nan    
        time_ml[ens_memb] = time_ml[ens_memb][~np.isnan(time_ml[ens_memb])]
        time[ens_memb] =(td[:,~np.isnan(snow_amount_ml[ens_memb]).any(axis=1)])
        height[ens_memb] = (h_above[~np.isnan(snow_amount_ml[ens_memb]).any(axis=1)])
        thickness_ml[ens_memb] = (thickness_ml[ens_memb][~np.isnan(snow_amount_ml[ens_memb]).any(axis=1),:])
        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb][~np.isnan(snow_amount_ml[ens_memb]).any(axis=1)])
# convert the snow amount from [kg/m^2] into [g/m^2] by multiply with 1000
        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb]*1000.)
#        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb]*1000.)/thickness_ml[ens_memb]

# find the maximum of all ensemble members
        if len(snow_amount_ml[ens_memb])  == 0:
            maxi = np.nan
        else:
            maxi = snow_amount_ml[ens_memb].max()
        maxim.append(maxi)
    maxim = np.asarray(maxim)
    print('%s. %s:' %(ini_day, calmon), np.nanmax(maxim))

    var_name_ml = var_name_ml.split("_")



    
    title = 'initialized: %s, %s %s %s 0%s UTC' %( calday,ini_day,calmon,year,hh)

    Xmax = time[0].max()
    pvert.plot_vertical_EM0_1(time, height, snow_amount_ml, time_ml[0], var_name_ml, unit, maxim, Xmax, title)
    fig_name = '%s%s%s_0%s_EM0_1.%s' %(year,month,ini_day,hh,form)
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
    else:
        plt.show()
    plt.close()

    pvert.plot_vertical_EM0_9(time, height, snow_amount_ml, time_ml[0], var_name_ml, unit, maxim, title)
    fig_name = '%s%s%s_0%s_EM0_9.%s' %(year,month,ini_day,hh,form)
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
    else:
        plt.show()
    plt.close

    Xmax = 48.
    pvert.plot_vertical_EM0_9_48h(time, height, snow_amount_ml, time_ml[0], var_name_ml, unit, maxim, Xmax, title)
    fig_name = '%s%s%s_0%s_EM09_48h.%s' %(year,month,ini_day,hh,form)
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
    else:
        plt.show()
        
    plt.close()

In [ ]:
########### divided by thickness plots #################
    maxim = []
    for ens_memb in range(0,10):
        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb]*1000.)/thickness_ml[ens_memb]
# find the maximum of all ensemble members when divided by thickness
        if len(snow_amount_ml[ens_memb])  == 0:
            maxi = np.nan
        else:
            maxi = np.nanmax(np.asarray(snow_amount_ml[ens_memb]))
        maxim.append(maxi)
#    maxim = np.asarray(maxim)
    print('%s. %s  (thickness):' %(ini_day, calmon), np.nanmax(maxim))
    
    
    title = 'initialized: %s, %s %s %s 0%s UTC' %( calday,ini_day,calmon,year,hh)
    var_name_ml = var_name_ml.split("_")
# change unit to g/m^3    
    unit = unit = '[g/m$^3$]'
    
    Xmax = time[0].max()
    pvert.plot_vertical_EM0_1(time, height, snow_amount_ml, time_ml[0], var_name_ml, unit, maxim, Xmax, title)
    fig_name = '%s%s%s_0%s_EM0_1_thickness.png' %(year,month,ini_day,hh)
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
    else:
        plt.show()
    plt.close()

    pvert.plot_vertical_EM0_9(time, height, snow_amount_ml, time_ml[0], var_name_ml, unit, maxim, title)
    fig_name = '%s%s%s_0%s_EM0_9_thickness.png' %(year,month,ini_day,hh)
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
    else:
        plt.show()
    plt.close

    Xmax = 48.
    pvert.plot_vertical_EM0_9_48h(time, height, snow_amount_ml, time_ml[0], var_name_ml, unit, maxim, Xmax, title)
    fig_name = '%s%s%s_0%s_EM09_48h_thickness.png' %(year,month,ini_day,hh)
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
    else:
        plt.show()
        
    plt.close()
    
    for ens_memb in range(0,10):
        fn_ml[ens_memb].close()